<a href="https://colab.research.google.com/github/mukaseevru/ds-school/blob/main/lesson%206/Add_column_without_OVER().ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import sqlite3
import os
path = "./drive/My Drive/my_databases"
if not os.path.exists(path):
    os.mkdir(path)
conn = sqlite3.connect(path + '/test.db')
print("Opened database successfully");

Opened database successfully


In [3]:
cursor = conn.cursor()
cursor.executescript('''
CREATE TABLE team_data(code integer PRIMARY KEY,
                                     team text, 
                                     country text, 
                                     season integer, 
                                     total_goals integer);''')
conn.commit()

In [4]:
cursor.execute("INSERT INTO team_data VALUES(1, 'Real Madrid', 'Spain', 2019, 53);")
cursor.execute("INSERT INTO team_data VALUES(2, 'Barcelona', 'Spain', 2019, 47);")
sections = [(3, 'Arsenal', 'UK', 2019, 52), (4, 'Real Madrid', 'Spain', 2018, 49),
            (5, 'Barcelona', 'Spain', 2018, 45), (6, 'Arsenal', 'UK', 2018, 50 )]
cursor.executemany("INSERT INTO team_data VALUES (?, ?, ?, ?, ?)", sections)

In [5]:
sql  = ''' SELECT * FROM team_data;'''
cursor.execute(sql)
for row in cursor:
    print(row)

(1, 'Real Madrid', 'Spain', 2019, 53)
(2, 'Barcelona', 'Spain', 2019, 47)
(3, 'Arsenal', 'UK', 2019, 52)
(4, 'Real Madrid', 'Spain', 2018, 49)
(5, 'Barcelona', 'Spain', 2018, 45)
(6, 'Arsenal', 'UK', 2018, 50)


In [6]:
sql = '''
-- Создаем временную таблицу и добавляем столбец с количеством строк
CREATE TABLE IF NOT EXISTS team_data_temp
(code integer PRIMARY KEY,
team text, 
country text, 
season integer, 
total_goals integer,
count_str number
);

-- Заполняем временную таблицу
INSERT INTO team_data_temp(code, team, country, season, total_goals, count_str)
SELECT code, team, country, season, total_goals, (SELECT COUNT(*) AS count_str FROM team_data) FROM team_data;

-- Удаляем старую таблицу
DROP TABLE team_data;

-- Переименовываем новую
ALTER TABLE team_data_temp RENAME TO team_data;
'''
cursor.executescript(sql)
for row in cursor:
  print(row)

In [7]:
sql  = '''SELECT * FROM team_data;'''
cursor.execute(sql)
for row in cursor:
    print(row)

(1, 'Real Madrid', 'Spain', 2019, 53, 6)
(2, 'Barcelona', 'Spain', 2019, 47, 6)
(3, 'Arsenal', 'UK', 2019, 52, 6)
(4, 'Real Madrid', 'Spain', 2018, 49, 6)
(5, 'Barcelona', 'Spain', 2018, 45, 6)
(6, 'Arsenal', 'UK', 2018, 50, 6)


In [8]:
sql  = '''DROP TABLE team_data;'''
cursor.execute(sql)
for row in cursor:
    print(row)
cursor.close()
conn.commit()
conn.close()